Este exemplo é baseado nos tutoriais disponíveis na página do TensorFlow (https://www.tensorflow.org/get_started/mnist/beginners e https://www.tensorflow.org/get_started/mnist/pros)
e também no TensorBoard https://www.youtube.com/watch?v=eBbEDRsCmv4 

In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
#import TensorFlow and MNIST
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

# carrega os dados do MNIST com os labels no formato "one-hot vector"
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [20]:
# definimos algumas constantes
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
IMAGE_CHANNELS = 1
IMAGE_SIZE = IMAGE_WIDTH * IMAGE_HEIGHT * IMAGE_CHANNELS

N_CLASSES = 10
MAX_ITERS = 201
BATCH_SIZE = 64
LOGDIR='/home/vitor/mnist/'
if not os.path.exists(LOGDIR):
    os.makedirs(LOGDIR)
n_samples = 1024

In [21]:
tf.reset_default_graph()
sess = tf.Session()
with tf.name_scope('input'):
    # um placeholder para entradas (imagens) shape=(BATCH_SIZE, n_features)
    x = tf.placeholder(tf.float32, shape=[None, IMAGE_SIZE], name='images')
    # um placeholder para os labels, shape = (BATCH_SIZE, n_classes)
    y = tf.placeholder(tf.float32, shape=[None, N_CLASSES], name="labels")

    # pesos W
    W = tf.Variable(
        tf.truncated_normal([IMAGE_SIZE, N_CLASSES], stddev=0.05),
        name='Weights')
    # viéses b
    b = tf.Variable(tf.zeros([N_CLASSES]), name='Biases')

    tf.summary.histogram("weights", W)
    tf.summary.histogram("biases", b)

    # modelo linear
    logits = tf.matmul(x, W) + b
    
with tf.name_scope('loss'):
    loss = loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    tf.summary.scalar("loss", loss)

with tf.name_scope("train"):
    # SGD + momentum para otimização
    learning_rate = 0.01
    momentum = 0.9
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    # minimiza o valor da loss
    optimizer = optimizer.minimize(loss)
    
with tf.name_scope("test_accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    test_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("test_accuracy", test_accuracy)

summ = tf.summary.merge_all()    
    
# inicializa as variáveis globais
sess.run(tf.global_variables_initializer())
writer = tf.summary.FileWriter(LOGDIR + "linear_regression")
writer.add_graph(sess.graph)
for iteration in range(MAX_ITERS):
    # obtém o próximo batch de imagens e labels
    (images, labels) = mnist.train.next_batch(BATCH_SIZE)
    
    # executa uma iteração da otimização
    (_, iter_loss) = sess.run([optimizer, loss], 
                          feed_dict={x: images,
                                     y: labels}) 
    
    # avalia o modelo a cada 10 iterações
    if iteration % 10 == 0:
        [test_acc, s] = sess.run([test_accuracy, summ], 
                                 feed_dict={x: mnist.test.images[:n_samples], 
                                            y: mnist.test.labels[:n_samples]})
        print('Iteration {}, loss = {}.'.format(iteration, iter_loss))
        print('Test Accuracy: {0}'.format(test_acc))
        writer.add_summary(s, iteration)

Iteration 0, loss = 2.4436404705047607.
Test Accuracy: 0.1083984375
Iteration 10, loss = 2.040421962738037.
Test Accuracy: 0.396484375
Iteration 20, loss = 1.5820215940475464.
Test Accuracy: 0.6484375
Iteration 30, loss = 1.2205810546875.
Test Accuracy: 0.7060546875
Iteration 40, loss = 0.8770782947540283.
Test Accuracy: 0.76171875
Iteration 50, loss = 0.8602623343467712.
Test Accuracy: 0.787109375
Iteration 60, loss = 0.5099860429763794.
Test Accuracy: 0.798828125
Iteration 70, loss = 0.6948658227920532.
Test Accuracy: 0.8193359375
Iteration 80, loss = 0.4978371858596802.
Test Accuracy: 0.8232421875
Iteration 90, loss = 0.6322829723358154.
Test Accuracy: 0.81640625
Iteration 100, loss = 0.668944239616394.
Test Accuracy: 0.82421875
Iteration 110, loss = 0.6026045083999634.
Test Accuracy: 0.8359375
Iteration 120, loss = 0.6109832525253296.
Test Accuracy: 0.8388671875
Iteration 130, loss = 0.5514481067657471.
Test Accuracy: 0.8291015625
Iteration 140, loss = 0.4958510994911194.
Test Accu

In [24]:
# Adiciona Convolução
def conv_layer(input, size_in, size_out, name="conv", name2="pool"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.05), name="Weights")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="Biases")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        relu = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", relu)

        return relu

# Adiciona Max Pooling
def max_pool(input, name):
    with tf.name_scope(name):
        return tf.nn.max_pool(input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# Adiciona totalmente conectada
def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.05), name="Weights")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="Biases")
        relu = tf.nn.relu(tf.matmul(input, w) + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", relu)

        return relu


tf.reset_default_graph()
sess = tf.Session()

# Setup placeholders
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, 784], name="images")
    y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

# Redimensiona a imagem
with tf.name_scope('input_reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 10)

# convoluções e max poolings
conv1 = conv_layer(x_image, 1, 32, "conv1", "pool1")
conv1 = max_pool(conv1, "pool1")
conv2 = conv_layer(conv1, 32, 64, "conv2", "pool2")
conv2 = max_pool(conv2, "pool2")

# transforma a imagem em um array
with tf.name_scope('flatten'):
    flattened = tf.reshape(conv2, [-1, 7 * 7 * 64])

# totalmente conectada 1
fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, "fc1")

with tf.name_scope('dropout'):
    # dropout: elimina algumas unidades; veremos no final se der tempo
    keep_prob = tf.placeholder(tf.float32)
    fc1 = tf.nn.dropout(fc1, keep_prob)

# totalmente conectada 2
fc2 = fc_layer(fc1, 1024, 10, "fc2")

with tf.name_scope("loss"):
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=fc2, labels=y), name="cross_entropy")
    tf.summary.scalar("loss", loss)


with tf.name_scope("train"):
    # SGD + momentum para otimização
    learning_rate = 0.01
    momentum = 0.9
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    # minimiza o valor da loss
    optimizer = optimizer.minimize(loss)

with tf.name_scope("test_accuracy"):
    correct_prediction = tf.equal(tf.argmax(fc2, 1), tf.argmax(y, 1))
    test_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("test_accuracy", test_accuracy)

summ = tf.summary.merge_all()    
    
# inicializa as variáveis globais
sess.run(tf.global_variables_initializer())

# iniciar logger
writer = tf.summary.FileWriter(LOGDIR + 'cnn')
writer.add_graph(sess.graph)

for iteration in range(MAX_ITERS):
    # obtém o próximo batch de imagens e labels
    (images, labels) = mnist.train.next_batch(BATCH_SIZE)
    
    # executa uma iteração da otimização
    (_, iter_loss) = sess.run([optimizer, loss], 
                          feed_dict={x: images,
                                     y: labels,
                                     keep_prob: 0.5})  
    
    # avalia o modelo a cada 10 iterações
    if iteration % 10 == 0:
        [test_acc, s] = sess.run([test_accuracy, summ], 
                                 feed_dict={x: mnist.test.images[:n_samples], 
                                            y: mnist.test.labels[:n_samples],
                                            keep_prob: 1.0})
        print('Iteration {}, loss = {}.'.format(iteration, iter_loss))
        print('Test Accuracy: {0}'.format(test_acc))
        writer.add_summary(s, iteration)

Iteration 0, loss = 2.317106008529663.
Test Accuracy: 0.12890625
Iteration 10, loss = 2.3016648292541504.
Test Accuracy: 0.1748046875
Iteration 20, loss = 2.2366647720336914.
Test Accuracy: 0.2099609375
Iteration 30, loss = 2.1758460998535156.
Test Accuracy: 0.3154296875
Iteration 40, loss = 1.996772289276123.
Test Accuracy: 0.4169921875
Iteration 50, loss = 1.568583607673645.
Test Accuracy: 0.4658203125
Iteration 60, loss = 1.5448983907699585.
Test Accuracy: 0.4833984375
Iteration 70, loss = 1.195321798324585.
Test Accuracy: 0.4921875
Iteration 80, loss = 1.36623215675354.
Test Accuracy: 0.4912109375
Iteration 90, loss = 1.4441382884979248.
Test Accuracy: 0.5751953125
Iteration 100, loss = 1.1648666858673096.
Test Accuracy: 0.6337890625
Iteration 110, loss = 0.7468717694282532.
Test Accuracy: 0.80078125
Iteration 120, loss = 0.6994645595550537.
Test Accuracy: 0.82421875
Iteration 130, loss = 0.823643147945404.
Test Accuracy: 0.845703125
Iteration 140, loss = 0.518156886100769.
Test Ac